In [13]:
import os, re
import pandas as pd
import ipywidgets as widgets
from collections import defaultdict

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.plotting import figure
from bokeh.charts import Bar
from IPython.display import display, clear_output
from IPython.display import clear_output

from bokeh.models import HoverTool, BoxSelectTool
from bokeh.layouts import row

output_notebook()

Loading BokehJS ...

In [14]:

class plotLineInfo(object):
    
    def __init__(self, baseDirs):
        self.dirs = widgets.Dropdown(options=baseDirs, description='dir:', layout=widgets.Layout(width='1024px'))
        self.csvs = widgets.Dropdown(options=self.setCsvFileChoice(), description='csv:', layout=widgets.Layout(width='1024px'))
        self.color = widgets.ColorPicker(concise=False, description='color', value='blue')
        self.dirs.observe(self.benchSelection)
        display(self.dirs, self.csvs, self.color)   

        
    def benchSelection(self, change):
        self.csvs.options=self.setCsvFileChoice()
        
        
    def setCsvFileChoice(self):
        csvFiles = set()
        for dirpath, dirnames, filenames in os.walk( self.dirs.value ):
            for fileName in filenames:
                if fileName.endswith(".csv"):
                    csvFiles.add(fileName)
        return sorted(list(csvFiles))

    
    def getCsvFilePaths(self):
        csvFiles = set()
        for dirpath, dirnames, filenames in os.walk( self.dirs.value ):
            for fileName in filenames:
                if fileName.endswith(self.csvs.value):
                    csvFiles.add(dirpath+"/"+fileName)
        return sorted(list(csvFiles))

    
    def getDataFrames(self):
        dataFrames=[]
        for file in self.getCsvFilePaths():
            dataFrames.append(pd.read_csv(file))
        return dataFrames
                
        
    def _ipython_display_(self, **kwargs):
        self.widget._ipython_display_(**kwargs)



In [15]:

class PerfCharts(object):
    
    def __init__(self):
        self.throughput = figure(title="throughput", width = 1024, height = 300, tools="pan,wheel_zoom,box_zoom,reset,resize,previewsave")    
        self.latency = figure(title="latency", width = 1024, height = 300, tools="pan,wheel_zoom,box_zoom,reset,resize,previewsave")
        self.count = figure(title="opp count", width = 1024, height = 300, tools="pan,wheel_zoom,box_zoom,reset,resize,previewsave")

        self.throughput.xaxis.axis_label = "time"
        self.throughput.yaxis.axis_label = "opeartions/sec"
        self.throughput.yaxis.major_label_orientation = "vertical"

        self.latency.xaxis.axis_label = "time"
        self.latency.yaxis.axis_label = "milliseconds"
        self.latency.yaxis.major_label_orientation = "vertical"
        
        self.count.xaxis.axis_label = "time"
        self.count.yaxis.axis_label = "op count"
        self.count.yaxis.major_label_orientation = "vertical"
        
        self.latency.add_tools(BoxSelectTool())
        
    def chart(self, plotInfo):
        for f in plotInfo.getDataFrames():
            f.reset_index(inplace = True)
            
            #self.throughput.line(source=f, x=f.columns[0], y=f.columns[14], line_color=plotInfo.color.value)   
            #self.latency.line(source=f, x=f.columns[0], y=f.columns[11], line_color=plotInfo.color.value)
            self.count.line(source=f, x=f.columns[0], y=f.columns[2], line_color=plotInfo.color.value)
            
    def show(self):
        show(self.throughput)
        show(self.latency)
        show(self.count)
        
    def clear_output(self):
        clear_output()

        

In [16]:

class AppGui(object):

    def __init__(self):
        self.plotlines=[]
        self.baseDirs = self.getCsvDirectorys()
        
        self.add = widgets.Button(description='add', button_style='success', tooltip='add to plot')
        self.add.on_click(self.onAdd)
        display(self.add)

        self.chart = widgets.Button(description='chart', button_style='success', tooltip='chart csv')
        self.chart.on_click(self.onChart)
        display(self.chart)

        self.perfCharts = PerfCharts()
        
    def onAdd(self, change):
        self.plotlines.append( plotLineInfo(self.baseDirs) )

    
    def onChart(self, change):
        for plotline in self.plotlines:
            self.perfCharts.chart(plotline)
        self.perfCharts.clear_output()
        self.perfCharts.show()
           
        
    def getCsvDirectorys(self):
        baseDirs = set()
        for dirpath, dirnames, filenames in os.walk("."):
            for fileName in filenames:
                if fileName.endswith(".csv"):
                    relDir = os.path.relpath(dirpath, ".")
                    baseDirs.add(re.sub('[^/]*$', '', relDir))
        return sorted(list(baseDirs))
    
    

In [17]:

AppGui()

In [18]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))